# This notebook is intended to solve an old problem for the Data Incubator.

### The problem is as follows:

A chess knight piece is sitting on the "0" key of  a numeric keypad.

1	2	3  
4	5	6  
7	8	9  
&nbsp;&nbsp;    0	 
 

 


The knight makes T jumps to other keys according to its allowable moves (so that from each reachable key it has two or three allowable moves). The knight chooses amongst the allowable moves uniformly at random and keeps track of the running sum SS of keys on which it lands. We are interested in S under the modulo operator.

After T=10T10 moves, what is the expected value of the quantity S mod 10?

In [1]:
import random
import time
from decimal import *

In [2]:
jumptable = {0:[4,6],1:[6,8],2:[7,9],3:[4,8],4:[0,3,9],5:[],6:[0,1,7],7:[2,6],8:[1,3],9:[2,4]}

In [79]:
def jump(steps):
    out = 0
    p = 1
    for i in range(steps):
        p *= Decimal(1)/Decimal(len(jumptable[out]))
        out = random.choice(jumptable[out])
    return [out,p]

In [80]:
def jump2(steps):
    out = 0
    p = Decimal(1)
    path = [0]
    for i in range(steps):
        p /= Decimal(len(jumptable[out]))
        out = random.choice(jumptable[out])
        path.append(out)
    return [sum(path)%10,p]

In [81]:
st=time.time()
print(jump(10))
print(time.time()-st)

[7, Decimal('0.0004340277777777777777777777775')]
0.00036787986755371094


In [82]:
st=time.time()
print(jump2(10))
print(time.time()-st)

[6, Decimal('0.000289351851851851851851851852')]
0.0002892017364501953


## This is a sloppy way to estimate the expectation

It should only work for very small values of steps.
Really really don't try large values of steps here. 20 steps took ~10 minutes

It's amazing how quickly you can make unworkable code.

In [83]:
def estavg(steps):#returns the average and the execution time
    psum = 0
    resultlist = []
    st = time.time()
    while psum < 1:
        a = jump(steps)
        resultlist.append(a[0])
        psum += a[1]
    return sum(resultlist)/len(resultlist), time.time()-st

In [86]:
def estexp(steps):#returns the average and the execution time
    psum = 0
    resultlist = 0
    st = time.time()
    while psum < 1:
        a = jump2(steps)
        resultlist += a[0]*a[1]
        psum += a[1]
    return resultlist, time.time()-st

In [92]:
avgexp = 0
iterations = 200
for i in range(iterations):
    avgexp += estexp(10)[0]
print(avgexp/iterations)

4.617353354873971193415637762


## Let's try something a bit more methodical/mathematical

Also apparently there was a reading comprehension failure. We care about the expectation of the modulo sum.

The expectation comes from the sum of all "paths" the knight can take.

In [12]:
def jumprecord(steps):
    out = 0
    p = 1
    record=[]
    for i in range(steps):
        p *= Decimal(1)/Decimal(len(jumptable[out]))
        out = random.choice(jumptable[out])
        record.append(out)
    return sum(record)%10,record

In [13]:
def pathstep(pathlist):#paths should be a list of paths
    newpathlist = []
    for path in pathlist:
        for jump in jumptable[path[-1]]:
            newpath = path.copy()
            newpath.append(jump)
            newpathlist.append((newpath))
    return newpathlist

In [51]:
def findexp(path):
    prob = Decimal(1)
    pathcopy = path.copy()
    pathcopy.pop()
    while pathcopy:
        if pathcopy[-1] == 4 or pathcopy[-1] == 6:
            prob /= Decimal(3)
        else:
            prob /= Decimal(2)
        pathcopy.pop()
    #print(sum(path),len(path))
    summod = sum(path)%10
    return summod*prob

In [61]:
paths = [[0]]
for i in range(17):
    paths = pathstep(paths)
#print(paths)

In [62]:
expectation = 0
for i in paths:
    expectation += findexp(i)
print(expectation)

4.475802802545343697607061862


## Is it possible to make something more scalable?

Finding all the paths for a number of jumps greater than ~15 starts to slow down a lot. Is it possible to keep a running record of the expectation?

The brief I read was unclear on the number of steps/jumps that the knight should take. If 10 steps is enough, then this is solved. If 10000 steps are needed, then a fundamentally different way of thinking about the problem will be necessary.